In [ ]:
!pip install nvidia-physicsnemo[all]

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 90.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.1/196.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.4/118.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch

import physicsnemo
from physicsnemo.datapipes.benchmarks.darcy import Darcy2D
from physicsnemo.metrics.general.mse import mse
from physicsnemo.models.fno.fno import FNO

Warp 1.7.2 initialized:
   CUDA Toolkit 12.8, Driver 12.4
   Devices:
     "cpu"      : "x86_64"
     "cuda:0"   : "Tesla T4" (15 GiB, sm_75, mempool enabled)
   Kernel cache:
     /root/.cache/warp/1.7.2


In [ ]:
normaliser = {
    "permeability": (1.25, 0.75),
    "darcy": (4.52e-2, 2.79e-2),
}
dataloader = Darcy2D(
    resolution=256, batch_size=64, nr_permeability_freq=5, normaliser=normaliser
)
model = FNO(
    in_channels=1,
    out_channels=1,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=4,
    num_fno_modes=12,
    padding=5,
).to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=lambda step: 0.85**step
)

# run for 20 iterations
for i in range(20):
    batch = next(iter(dataloader))
    true = batch["darcy"]
    pred = model(batch["permeability"])
    loss = mse(pred, true)
    loss.backward()
    optimizer.step()
    scheduler.step()

    print(f"Iteration:{i}. Loss:{loss.detach().cpu().numpy()}")

Module physicsnemo.datapipes.benchmarks.kernels.initialization 2b6fc90 load on device 'cuda:0' took 525.73 ms  (compiled)
Module physicsnemo.datapipes.benchmarks.kernels.utils f973d89 load on device 'cuda:0' took 931.29 ms  (compiled)
Module physicsnemo.datapipes.benchmarks.kernels.finite_difference 1996b0e load on device 'cuda:0' took 566.69 ms  (compiled)
Iteration:0. Loss:0.6778674721717834
Iteration:1. Loss:1.2777646780014038
Iteration:2. Loss:1.023526906967163
Iteration:3. Loss:0.7406870126724243
Iteration:4. Loss:0.8542118668556213
Iteration:5. Loss:0.5971039533615112
Iteration:6. Loss:1.2102512121200562
Iteration:7. Loss:0.5211585760116577
Iteration:8. Loss:0.5480004549026489
Iteration:9. Loss:0.5664846301078796
Iteration:10. Loss:0.4080187678337097
Iteration:11. Loss:0.3260185718536377
Iteration:12. Loss:0.2695801854133606
Iteration:13. Loss:0.27908656001091003
Iteration:14. Loss:0.3597103953361511
Iteration:15. Loss:0.4145232141017914
Iteration:16. Loss:0.5844050049781799
Iter

## WandBでログをとるバージョン

In [ ]:
from physicsnemo.launch.logging import LaunchLogger, PythonLogger
from physicsnemo.launch.logging.wandb import initialize_wandb

In [ ]:
normaliser = {
    "permeability": (1.25, 0.75),
    "darcy": (4.52e-2, 2.79e-2),
}
dataloader = Darcy2D(
    resolution=256, batch_size=64, nr_permeability_freq=5, normaliser=normaliser
)
model = FNO(
    in_channels=1,
    out_channels=1,
    decoder_layers=1,
    decoder_layer_size=32,
    dimension=2,
    latent_channels=32,
    num_fno_layers=4,
    num_fno_modes=12,
    padding=5,
).to("cuda")

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.LambdaLR(
    optimizer, lr_lambda=lambda step: 0.85**step
)

# Initialize the console logger
logger = PythonLogger("main")  # General python logger

# Initialize the MLFlow logger
initialize_wandb(
    project="PhysicsNeMo Tutorials",
    name="Simple PhysicsNeMo Tutorials",
    entity="PhysicsNeMo MLFLow Tutorial",
    mode="offline",
)
LaunchLogger.initialize(use_wandb=True)

# Use logger methods to track various information during training
logger.info("Starting Training!")

# we will setup the training to run for 20 epochs each epoch running for 10 iterations
for i in range(20):
    # wrap the epoch in launch logger to control frequency of output for console logs
    with LaunchLogger("train", epoch=i) as launchlog:
        # this would be iterations through different batches
        for _ in range(10):
            batch = next(iter(dataloader))
            true = batch["darcy"]
            pred = model(batch["permeability"])
            loss = mse(pred, true)
            loss.backward()
            optimizer.step()
            scheduler.step()
            launchlog.log_minibatch({"Loss": loss.detach().cpu().numpy()})

        launchlog.log_epoch({"Learning Rate": optimizer.param_groups[0]["lr"]})


logger.info("Finished Training!")

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


Module physicsnemo.datapipes.benchmarks.kernels.initialization 2b6fc90 load on device 'cuda:0' took 931.45 ms  (compiled)
Module physicsnemo.datapipes.benchmarks.kernels.utils f973d89 load on device 'cuda:0' took 2311.82 ms  (compiled)
Module physicsnemo.datapipes.benchmarks.kernels.finite_difference 1996b0e load on device 'cuda:0' took 1158.38 ms  (compiled)
